### This Apache Beam pipeline performs a computation of the average value from a collection of numbers and writes the result to a text file. Let’s break down each part:

#### 1. Pipeline Initialization

a. Imports the Apache Beam library.

b. Initializes a Beam pipeline, which is the primary construct for defining and executing a data processing workflow.

In [1]:
import apache_beam as beam

p = beam.Pipeline()



##### 2. Define a Custom CombineFn for Average Calculation

AverageFn is a custom CombineFn (a Beam class for combining values) that calculates the average of a collection of numbers.

    1. create_accumulator(self): Initializes the accumulator as a tuple (sum, count), where sum is 0.0 and count is 0.

    2. add_input(self, sum_count, input): Updates the accumulator by adding the new input to the sum and incrementing the count.

    3. merge_accumulators(self, accumulators): Merges multiple accumulators from parallel processing. It sums up all individual sums and counts.

    4. extract_output(self, sum_count): Computes the average by dividing sum by count. Returns NaN if the count is zero.

In [ ]:
class AverageFn(beam.CombineFn):
  
  def create_accumulator(self):
     return (0.0, 0)   # initialize (sum, count)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):
    
    ind_sums, ind_counts = zip(*accumulators)       # zip - [(27, 3), (39, 3), (18, 2)]  -->   [(27,39,18), (3,3,2)]
    return sum(ind_sums), sum(ind_counts)        # (84,8)

  def extract_output(self, sum_count):    
    
    (sum, count) = sum_count    # combine globally using CombineFn
    return sum / count if count else float('NaN')

##### 3. Apply the CombineGlobally Transformation

1. beam.Create([15,5,7,7,9,23,13,5]): Creates a PCollection (a distributed collection of data) from the list of numbers.

2. "Combine Globally" >> beam.CombineGlobally(AverageFn()): Applies the AverageFn to the entire PCollection to compute the average. CombineGlobally is used to aggregate the entire collection into a single result.

3. 'Write results' >> beam.io.WriteToText('Result/combine'): Writes the result of the computation to a text file named 'data/combine'.

In [2]:
small_sum = (
           p 
            | beam.Create([15,5,7,7,9,23,13,5])
            | "Combine Globally" >> beam.CombineGlobally(AverageFn()) 
            | 'Write results' >> beam.io.WriteToText('Result/combine')
          )
p.run()